<a href="https://colab.research.google.com/github/blindsphynx/Diploma/blob/main/ruGPT3_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка зависимостей

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
# репозиторий Сбера, который содержит модель и скрипт токенизации
!git clone  https://github.com/sberbank-ai/ru-gpts

# Файн-тьюнинг

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py

In [ ]:
!pip install --upgrade accelerate

In [ ]:
# запустить ячейку при ошибке фрагментации памяти (при необходимости поменять значение max_split_size_mb)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [ ]:
files = ["train_gpt.txt", "valid_gpt.txt"]

for f in files:
  file = open(f, "r", encoding="utf-8")
  text = file.read()
  res = ""
  res += text.encode(encoding="utf-8", errors="ignore")
  file.close()

  newfile = open(f, "w", encoding="utf-8")
  newfile.write(res)
  newfile.close()

In [ ]:
!python run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file train_gpt.txt \
    --validation_file valid_gpt.txt \
    --num_train_epochs=13.0 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --block_size 2048 \
    --dataset_config_name plain_text \
    --do_train \
    --do_eval \
    --output_dir drive/MyDrive/ruGPT3_new \
    --overwrite_output_dir

In [ ]:
import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)

# Генерация

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

In [5]:
def generate(
    model, tok, text,
    do_sample=True, max_length=50, min_length=5, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      min_length=min_length,
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [ ]:
tok, model = load_tokenizer_and_model("drive/MyDrive/ruGPT-3_small_conversational/")
query = input()
text = f"Вопрос: {query} Ответ: "
generated = generate(model, tok, text, num_beams=10) 

response = generated[0].split("Ответ:")[1].split("\n")[0]
print("Я: ", query)
print("Модель: ", response)